In [1]:
import pandas as pd

In [2]:
def selection(df, column_name, value):
    """
    Selects rows from the DataFrame where the specified column matches the given value.
    """
    return df[df[column_name] == value]


In [3]:
def convert_units(df, value_column, unit_column, mw_column):
    """
    Converts the values in the DataFrame from nM and µg/mL to µM based on their units.
    """
    # Conversion factors
    nm_to_um_factor = 1000  # 1 nM = 0.001 µM
    
    def convert_value(row):
        value = float(row[value_column])
        unit = row[unit_column]
        mw = float(row[mw_column])

        if unit == 'nM':
            return value / nm_to_um_factor
        elif unit == 'ug.mL-1':
            return (value * 1000) / mw  # Convert µg/mL to µM using molecular weight
        else:
            return value  # If already in µM or another unit, no conversion

    # Apply the conversion to each row
    df[value_column] = df.apply(convert_value, axis=1)
    
    # Update the units column to reflect the new common unit
    df[unit_column] = 'µM'
    
    return df


In [4]:
def assign_activity(df, value_column, threshold, class1, class2):
    """
    Assigns activity classes based on the comparison of a column's value to a threshold.
    """
    # Define a function to assign the class based on the value
    def activity_class(value):
        if value > threshold:
            return class1
        else:
            return class2

    # Apply the function to the value column and create a new 'Activity' column
    df['Activity'] = df[value_column].apply(activity_class)
    
    return df


In [5]:
d1 = pd.read_csv('DOWNLOAD-0oIefQWj3uhG8Dv5kO_h6lFT61bLHWBxEUqGs9e5zW8=.csv', delimiter = ';')
d1.head()

Molecule ChEMBL ID  Molecule Name  Molecule Max Phase  Molecular Weight  \
0            CHEMBL8  CIPROFLOXACIN                 4.0            331.35   
1       CHEMBL515284            NaN                 NaN            325.74   
2       CHEMBL515581            NaN                 NaN            350.36   
3       CHEMBL454211            NaN                 NaN            290.30   
4       CHEMBL443404            NaN                 NaN            449.08   

   #RO5 Violations  AlogP      Compound Key  \
0                0   1.58  4, Ciprofloxacin   
1                0   2.56                4p   
2                0   2.53                4k   
3                0   2.51                4a   
4                0   3.43                4r   

                                              Smiles Standard Type  \
0         O=C(O)c1cn(C2CC2)c2cc(N3CCNCC3)c(F)cc2c1=O           MIC   
1      O=[N+]([O-])c1ccc(/C=N/NC(=S)Nc2ccc(Cl)cn2)o1           MIC   
2  COc1ccc(NC(=S)N/N=C/c2ccc([N+](=O)[O-])o2)c(OC)c1           MIC   
3          O=[N+]([O-])c1ccc(/C=N/NC(=S)Nc2ccccc2)o1           MIC   
4    O=[N+]([O-])c1ccc(/C=N/NC(=S)Nc2ncc(Br)cc2Br)o1           MIC   

  Standard Relation  ...  Target Type Document ChEMBL ID  Source ID  \
0               '='  ...     ORGANISM      CHEMBL1136842          1   
1               '='  ...     ORGANISM      CHEMBL1136842          1   
2               '='  ...     ORGANISM      CHEMBL1136842          1   
3               '='  ...     ORGANISM      CHEMBL1136842          1   
4               '='  ...     ORGANISM      CHEMBL1136842          1   

      Source Description      Document Journal Document Year  Cell ChEMBL ID  \
0  Scientific Literature  Bioorg Med Chem Lett          2009             NaN   
1  Scientific Literature  Bioorg Med Chem Lett          2009             NaN   
2  Scientific Literature  Bioorg Med Chem Lett          2009             NaN   
3  Scientific Literature  Bioorg Med Chem Lett          2009             NaN   
4  Scientific Literature  Bioorg Med Chem Lett          2009             NaN   

   Properties  Action Type  Standard Text Value  
0         NaN          NaN                  NaN  
1         NaN          NaN                  NaN  
2         NaN          NaN                  NaN  
3         NaN          NaN                  NaN  
4         NaN          NaN                  NaN  

[5 rows x 47 columns]

In [6]:
d1.shape

(21, 47)

In [7]:
d1 = d1[['Smiles', 'Molecular Weight', 'Standard Value', 'Standard Type', 'Standard Units']]
d1.head()

Smiles  Molecular Weight  \
0         O=C(O)c1cn(C2CC2)c2cc(N3CCNCC3)c(F)cc2c1=O            331.35   
1      O=[N+]([O-])c1ccc(/C=N/NC(=S)Nc2ccc(Cl)cn2)o1            325.74   
2  COc1ccc(NC(=S)N/N=C/c2ccc([N+](=O)[O-])o2)c(OC)c1            350.36   
3          O=[N+]([O-])c1ccc(/C=N/NC(=S)Nc2ccccc2)o1            290.30   
4    O=[N+]([O-])c1ccc(/C=N/NC(=S)Nc2ncc(Br)cc2Br)o1            449.08   

   Standard Value Standard Type Standard Units  
0          2350.0           MIC             nM  
1         38300.0           MIC             nM  
2         17800.0           MIC             nM  
3         86100.0           MIC             nM  
4          6900.0           MIC             nM

In [8]:
d1.shape

(21, 5)

In [9]:
d1 = selection(d1, 'Standard Type', 'MIC')
d1.shape

(21, 5)

In [10]:
d1.describe()

Molecular Weight  Standard Value
count         21.000000       21.000000
mean         347.522857    48254.761905
std          122.441954    36393.128695
min          137.140000     2350.000000
25%          305.320000    20500.000000
50%          320.330000    38400.000000
75%          350.360000    72100.000000
max          822.950000   157000.000000

In [11]:
d1 = convert_units(d1, 'Standard Value', 'Standard Units', 'Molecular Weight')
d1.shape

(21, 5)

In [12]:
d1.describe()

Molecular Weight  Standard Value
count         21.000000       21.000000
mean         347.522857       48.254762
std          122.441954       36.393129
min          137.140000        2.350000
25%          305.320000       20.500000
50%          320.330000       38.400000
75%          350.360000       72.100000
max          822.950000      157.000000

In [13]:
d1['Standard Units'].unique()

array(['µM'], dtype=object)

In [14]:
d1.drop_duplicates(subset = 'Smiles', inplace = True)
d1.shape

(21, 5)

In [15]:
d2 = pd.read_csv('DOWNLOAD-3BUsbBv8_gtkwqrEQlsiYcWb3GJfNLE2yimHKP_omso=.csv', delimiter = ';')
d2.head()

Molecule ChEMBL ID Molecule Name  Molecule Max Phase  Molecular Weight  \
0       CHEMBL392953           NaN                 NaN            492.91   
1       CHEMBL361984           NaN                 NaN            270.32   
2       CHEMBL235088           NaN                 NaN            445.33   
3      CHEMBL2435254           NaN                 NaN            318.30   
4      CHEMBL2435275           NaN                 NaN            308.34   

   #RO5 Violations  AlogP Compound Key  \
0              0.0   1.83           23   
1              0.0   2.02           18   
2              0.0   1.79           14   
3              0.0   3.97          19c   
4              0.0   4.51          17a   

                                              Smiles Standard Type  \
0  Cc1cn([C@@H]2C[C@H](O)[C@@H](CNC(=S)NCc3ccc(Cl...         MIC99   
1                CCCCCCCC[C@@H]1OC(=O)C(=O)C1C(=O)OC         MIC99   
2  Cc1cn([C@@H]2C[C@H](O)[C@@H](CNC(=S)Nc3ccc(Cl)...         MIC99   
3        O=C1c2ccccc2C(=O)c2c(O)cc(-c3ccc(F)cc3)cc21          GI50   
4            O=C1c2ccccc2-c2nccc3nc(-c4ccccc4)cc1c23          GI50   

  Standard Relation  ...  Target Type Document ChEMBL ID  Source ID  \
0               '='  ...     ORGANISM      CHEMBL1137559          1   
1               '>'  ...     ORGANISM      CHEMBL1142794          1   
2               '='  ...     ORGANISM      CHEMBL1137559          1   
3               '='  ...     ORGANISM      CHEMBL2434878          1   
4               '='  ...     ORGANISM      CHEMBL2434878          1   

      Source Description      Document Journal Document Year  Cell ChEMBL ID  \
0  Scientific Literature            J Med Chem        2007.0             NaN   
1  Scientific Literature  Bioorg Med Chem Lett        2005.0             NaN   
2  Scientific Literature            J Med Chem        2007.0             NaN   
3  Scientific Literature        Eur J Med Chem        2013.0             NaN   
4  Scientific Literature        Eur J Med Chem        2013.0             NaN   

   Properties  Action Type  Standard Text Value  
0         NaN          NaN                  NaN  
1         NaN          NaN                  NaN  
2         NaN          NaN                  NaN  
3         NaN          NaN                  NaN  
4         NaN          NaN                  NaN  

[5 rows x 47 columns]

In [16]:
d2.shape

(1746, 47)

In [17]:
d2 = d2[['Smiles', 'Molecular Weight', 'Standard Value', 'Standard Type', 'Standard Units']]
d2.head()

Smiles  Molecular Weight  \
0  Cc1cn([C@@H]2C[C@H](O)[C@@H](CNC(=S)NCc3ccc(Cl...            492.91   
1                CCCCCCCC[C@@H]1OC(=O)C(=O)C1C(=O)OC            270.32   
2  Cc1cn([C@@H]2C[C@H](O)[C@@H](CNC(=S)Nc3ccc(Cl)...            445.33   
3        O=C1c2ccccc2C(=O)c2c(O)cc(-c3ccc(F)cc3)cc21            318.30   
4            O=C1c2ccccc2-c2nccc3nc(-c4ccccc4)cc1c23            308.34   

   Standard Value Standard Type Standard Units  
0            40.0         MIC99        ug ml-1  
1            25.0         MIC99        ug ml-1  
2            25.0         MIC99        ug ml-1  
3           850.0          GI50             nM  
4           670.0          GI50             nM

In [18]:
d2.shape

(1746, 5)

In [19]:
d2 = selection(d2, 'Standard Type', 'MIC')
d2.shape

(405, 5)

In [20]:
d2.describe()

Molecular Weight  Standard Value
count        405.000000    4.010000e+02
mean         432.304938    1.689609e+04
std          201.187247    9.016686e+04
min          123.110000    3.000000e-03
25%          307.230000    1.560000e+00
50%          402.290000    2.949000e+01
75%          489.380000    2.000000e+03
max         1271.440000    1.268000e+06

In [21]:
d2 = convert_units(d2, 'Standard Value', 'Standard Units', 'Molecular Weight')
d2.shape

(405, 5)

In [22]:
d2.describe()

Molecular Weight  Standard Value
count        405.000000      401.000000
mean         432.304938      102.698888
std          201.187247      387.837037
min          123.110000        0.003500
25%          307.230000        2.000000
50%          402.290000       13.769394
75%          489.380000       62.500000
max         1271.440000     6498.253594

In [23]:
d2['Standard Units'].unique()

array(['µM'], dtype=object)

In [24]:
d2.drop_duplicates(subset = 'Smiles', inplace = True)
d2.shape

(300, 5)

In [25]:
d3  = pd.read_csv('DOWNLOAD-iqoIkaVjViZ1WCzRCmGcztk9psyvknrqgjwECbc51RQ=.csv', delimiter = ';')
d3.head()

Molecule ChEMBL ID Molecule Name  Molecule Max Phase  Molecular Weight  \
0      CHEMBL2097971           NaN                 NaN            352.39   
1       CHEMBL518923           NaN                 NaN            345.41   
2      CHEMBL2098172           NaN                 NaN            402.20   
3      CHEMBL3637828           NaN                 NaN            350.40   
4      CHEMBL3637811           NaN                 NaN            215.26   

   #RO5 Violations  AlogP  Compound Key  \
0                0   3.58    GSK358607A   
1                0   3.45    GSK888636A   
2                0   4.90   GSK1829736A   
3                0   3.94  TCMDC-143696   
4                0   2.42  TCMDC-143678   

                                              Smiles Standard Type  \
0            COc1ccc2nc(C)cc(OCC(=O)Nc3ccccc3OC)c2c1         MIC90   
1            COc1ccc(CCNc2cc(-c3ccccc3)nc3ncnn23)cc1         MIC90   
2  Cc1ccc2nc(C(F)(F)F)c(C(=O)NCc3cc(Cl)cc(Cl)c3)n2c1         MIC90   
3          Fc1cccc(Oc2cc(C3CCCNC3)nc(-c3ccncc3)n2)c1           MIC   
4                             COc1ccc(Nc2ncccc2N)cc1           MIC   

  Standard Relation  ...  Target Type Document ChEMBL ID  Source ID  \
0               '='  ...     ORGANISM      CHEMBL2095176         22   
1               '='  ...     ORGANISM      CHEMBL2095176         22   
2               '='  ...     ORGANISM      CHEMBL2095176         22   
3               '='  ...     ORGANISM      CHEMBL3637782         22   
4               '='  ...     ORGANISM      CHEMBL3637782         22   

           Source Description Document Journal Document Year  Cell ChEMBL ID  \
0  GSK Tuberculosis Screening              NaN           NaN             NaN   
1  GSK Tuberculosis Screening              NaN           NaN             NaN   
2  GSK Tuberculosis Screening              NaN           NaN             NaN   
3  GSK Tuberculosis Screening              NaN           NaN             NaN   
4  GSK Tuberculosis Screening              NaN           NaN             NaN   

   Properties  Action Type  Standard Text Value  
0         NaN          NaN                  NaN  
1         NaN          NaN                  NaN  
2         NaN          NaN                  NaN  
3         NaN          NaN                  NaN  
4         NaN          NaN                  NaN  

[5 rows x 47 columns]

In [26]:
d3.shape

(422, 47)

In [27]:
d3 = d3[['Smiles', 'Molecular Weight', 'Standard Value', 'Standard Type', 'Standard Units']]
d3.head()

Smiles  Molecular Weight  \
0            COc1ccc2nc(C)cc(OCC(=O)Nc3ccccc3OC)c2c1            352.39   
1            COc1ccc(CCNc2cc(-c3ccccc3)nc3ncnn23)cc1            345.41   
2  Cc1ccc2nc(C(F)(F)F)c(C(=O)NCc3cc(Cl)cc(Cl)c3)n2c1            402.20   
3          Fc1cccc(Oc2cc(C3CCCNC3)nc(-c3ccncc3)n2)c1            350.40   
4                             COc1ccc(Nc2ncccc2N)cc1            215.26   

   Standard Value Standard Type Standard Units  
0           400.0         MIC90             nM  
1           600.0         MIC90             nM  
2           900.0         MIC90             nM  
3         10000.0           MIC             nM  
4          5600.0           MIC             nM

In [28]:
d3.shape

(422, 5)

In [29]:
d3 = selection(d3, 'Standard Type', 'MIC')
d3.shape

(59, 5)

In [30]:
d3.describe()

Molecular Weight  Standard Value
count         59.000000       59.000000
mean         338.591356     5145.762712
std           65.407969     3623.133148
min          215.260000      200.000000
25%          303.395000     2500.000000
50%          343.380000     5000.000000
75%          377.440000     6750.000000
max          489.570000    20000.000000

In [31]:
d3 = convert_units(d3, 'Standard Value', 'Standard Units', 'Molecular Weight')
d3.shape

(59, 5)

In [32]:
d3.describe()

Molecular Weight  Standard Value
count         59.000000       59.000000
mean         338.591356        5.145763
std           65.407969        3.623133
min          215.260000        0.200000
25%          303.395000        2.500000
50%          343.380000        5.000000
75%          377.440000        6.750000
max          489.570000       20.000000

In [33]:
d3['Standard Units'].unique()

array(['µM'], dtype=object)

In [34]:
d3.drop_duplicates(subset = 'Smiles', inplace = True)
d3.shape

(50, 5)

In [35]:
d4 = pd.read_csv('DOWNLOAD-K3Z0sUtH4NKeiFIsfLz14t_qZfIxjaNtCj3hFgZMi08=.csv', delimiter = ';')
d4.head()

C:\Users\saatvik\AppData\Local\Temp\ipykernel_17960\3302153086.py:1: DtypeWarning: Columns (27,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  d4 = pd.read_csv('DOWNLOAD-K3Z0sUtH4NKeiFIsfLz14t_qZfIxjaNtCj3hFgZMi08=.csv', delimiter = ';')


Molecule ChEMBL ID Molecule Name  Molecule Max Phase  Molecular Weight  \
0      CHEMBL1829654           NaN                 NaN            365.34   
1      CHEMBL1829643           NaN                 NaN            367.45   
2        CHEMBL44884    ETHAMBUTOL                 4.0            204.31   
3       CHEMBL329377           NaN                 NaN            384.65   
4       CHEMBL374478      RIFAMPIN                 4.0            822.95   

   #RO5 Violations  AlogP Compound Key  \
0              1.0   5.48          13c   
1              1.0   5.11           7c   
2              0.0  -0.29          EMB   
3              1.0   6.58           7a   
4              3.0   4.34          RFP   

                                              Smiles Standard Type  \
0  O=[N+]([O-])c1cccc(-c2csc(Nc3ccccc3C(F)(F)F)n2)c1           MIC   
1         COc1cccc2c1CCC/C2=N/Nc1nc(-c2ccc(F)cc2)cs1           MIC   
2                       CC[C@@H](CO)NCCN[C@@H](CC)CO           MIC   
3               CCCCCCCCCCCCCCC(CNCC)NC(=O)OC(C)(C)C         Ratio   
4  CO[C@H]1/C=C/O[C@@]2(C)Oc3c(C)c(O)c4c(O)c(c(/C...           MIC   

  Standard Relation  ...  Target Type Document ChEMBL ID  Source ID  \
0               '>'  ...     ORGANISM      CHEMBL1828542          1   
1               '='  ...     ORGANISM      CHEMBL1828542          1   
2               '>'  ...     ORGANISM      CHEMBL1155191          1   
3               '>'  ...     ORGANISM      CHEMBL1155191          1   
4               '='  ...     ORGANISM      CHEMBL1122862          1   

      Source Description      Document Journal Document Year  Cell ChEMBL ID  \
0  Scientific Literature  Bioorg Med Chem Lett        2011.0             NaN   
1  Scientific Literature  Bioorg Med Chem Lett        2011.0             NaN   
2  Scientific Literature  Bioorg Med Chem Lett        2009.0             NaN   
3  Scientific Literature  Bioorg Med Chem Lett        2009.0             NaN   
4  Scientific Literature            J Med Chem        1985.0             NaN   

   Properties  Action Type  Standard Text Value  
0         NaN          NaN                  NaN  
1         NaN          NaN                  NaN  
2         NaN          NaN                  NaN  
3         NaN          NaN                  NaN  
4         NaN          NaN                  NaN  

[5 rows x 47 columns]

In [36]:
d4.shape

(203094, 47)

In [37]:
d4 = d4[['Smiles', 'Molecular Weight', 'Standard Value', 'Standard Type', 'Standard Units']]
d4.shape

(203094, 5)

In [38]:
d4 = selection(d4, 'Standard Type', 'MIC')
d4.shape

(44364, 5)

In [39]:
d4.describe()

Molecular Weight  Standard Value
count      44364.000000    4.279300e+04
mean         411.391022    7.263500e+04
std          180.231806    1.677123e+06
min           45.090000    0.000000e+00
25%          318.490000    6.250000e+00
50%          385.600000    1.000000e+02
75%          466.550000    8.060000e+03
max         2841.940000    7.669000e+07

In [40]:
d4 = convert_units(d4, 'Standard Value', 'Standard Units', 'Molecular Weight')
d4.shape

(44364, 5)

In [41]:
d4.describe()

Molecular Weight  Standard Value
count      44364.000000    42793.000000
mean         411.391022      129.950646
std          180.231806     2243.057109
min           45.090000        0.000000
25%          318.490000        1.880000
50%          385.600000       13.211785
75%          466.550000       58.700000
max         2841.940000   303785.163133

In [42]:
d4.head()

Smiles  Molecular Weight  \
0  O=[N+]([O-])c1cccc(-c2csc(Nc3ccccc3C(F)(F)F)n2)c1            365.34   
1         COc1cccc2c1CCC/C2=N/Nc1nc(-c2ccc(F)cc2)cs1            367.45   
2                       CC[C@@H](CO)NCCN[C@@H](CC)CO            204.31   
4  CO[C@H]1/C=C/O[C@@]2(C)Oc3c(C)c(O)c4c(O)c(c(/C...            822.95   
6  Clc1ccc(-c2nc3ccccc3c(-c3ccccc3)c2Sc2nc3ccccc3...            481.05   

   Standard Value Standard Type Standard Units  
0       34.214704           MIC             µM  
1        6.250000           MIC             µM  
2      158.000000           MIC             µM  
4        0.030379           MIC             µM  
6       12.900000           MIC             µM

In [43]:
d4['Standard Units'].unique()

array(['µM'], dtype=object)

In [44]:
d4.drop_duplicates(subset = 'Smiles', inplace = True)
d4.shape

(23522, 5)

In [45]:
d5 = pd.read_csv('DOWNLOAD-waOopUPMniOvhQOih6p5VBVqCQhsPYUhy44L8WxqLTk=.csv', delimiter = ';')
d5.head()

Molecule ChEMBL ID Molecule Name  Molecule Max Phase  Molecular Weight  \
0      CHEMBL2098272           NaN                 NaN            495.91   
1      CHEMBL2098277           NaN                 NaN            310.78   
2      CHEMBL2098291           NaN                 NaN            592.55   
3      CHEMBL1477639           NaN                 NaN            242.28   
4      CHEMBL2098475           NaN                 NaN            415.92   

   #RO5 Violations  AlogP Compound Key  \
0              0.0   3.78  SB-790652-M   
1              1.0   5.53    SB-657505   
2              0.0   3.40   GSK541718A   
3              0.0   3.02    GW560770X   
4              1.0   5.74   GSK498384A   

                                              Smiles Standard Type  \
0  O=C(Nc1ccccc1-c1nc(Cl)cs1)OCCOC1CCNCC1.O=C(O)C...         MIC90   
1                    Oc1ccc(Nc2ccccc2Nc2ccccc2)cc1Cl         MIC90   
2  CNC(=O)c1cc(Nc2ncc(F)c(N(C)c3ccc4c(C)[nH]nc4c3...         MIC90   
3                      Cc1ccc(Nc2ccc(C(=O)O)cc2N)cc1         MIC90   
4  Cc1cc(C)c2cc(CN(Cc3cccnc3)C(=O)c3ccccc3)c(Cl)n...         MIC90   

  Standard Relation  ...  Target Type Document ChEMBL ID  Source ID  \
0               '='  ...     ORGANISM      CHEMBL2095176         22   
1               '='  ...     ORGANISM      CHEMBL2095176         22   
2               '='  ...     ORGANISM      CHEMBL2095176         22   
3               '='  ...     ORGANISM      CHEMBL2095176         22   
4               '='  ...     ORGANISM      CHEMBL2095176         22   

           Source Description Document Journal Document Year  Cell ChEMBL ID  \
0  GSK Tuberculosis Screening              NaN           NaN             NaN   
1  GSK Tuberculosis Screening              NaN           NaN             NaN   
2  GSK Tuberculosis Screening              NaN           NaN             NaN   
3  GSK Tuberculosis Screening              NaN           NaN             NaN   
4  GSK Tuberculosis Screening              NaN           NaN             NaN   

   Properties  Action Type  Standard Text Value  
0         NaN          NaN                  NaN  
1         NaN          NaN                  NaN  
2         NaN          NaN                  NaN  
3         NaN          NaN                  NaN  
4         NaN          NaN                  NaN  

[5 rows x 47 columns]

In [46]:
d5.shape

(1626, 47)

In [47]:
d5 = d5[['Smiles', 'Molecular Weight', 'Standard Value', 'Standard Type', 'Standard Units']]
d5.shape

(1626, 5)

In [48]:
d5 = selection(d5, 'Standard Type', 'MIC')
d5.shape

(266, 5)

In [49]:
d5.describe()

Molecular Weight  Standard Value
count        266.000000      261.000000
mean         445.041241    29364.113243
std          256.780779   104212.869869
min          123.110000        0.000200
25%          334.400000        4.000000
50%          370.870000       30.000000
75%          463.320000      250.000000
max         1449.270000   661000.000000

In [50]:
d5 = convert_units(d5, 'Standard Value', 'Standard Units', 'Molecular Weight')
d5.shape

(266, 5)

In [51]:
d5.describe()

Molecular Weight  Standard Value
count        266.000000      261.000000
mean         445.041241      609.436342
std          256.780779     1575.332854
min          123.110000        0.000243
25%          334.400000        7.006596
50%          370.870000       72.741380
75%          463.320000      172.473267
max         1449.270000     5723.934633

In [52]:
d5['Standard Units'].unique()

array(['µM'], dtype=object)

In [53]:
d5.drop_duplicates(subset = 'Smiles', inplace = True)
d5.shape

(150, 5)

In [54]:
d6 = pd.read_csv('PubChem_compound_text_ANTITUBERCULAR.csv')
d6.head()

cid       cmpdname  \
0       6234    Cycloserine   
1  135398735       Rifampin   
2       3767      Isoniazid   
3       1046   Pyrazinamide   
4       2764  Ciprofloxacin   

                                         cmpdsynonym      mw           mf  \
0  D-cycloserine|cycloserine|68-41-7|Seromycin|or...  102.09     C3H6N2O2   
1  Rifampicin|rifampin|Rifadin|Rimactane|Rimactan...  822.90  C43H58N4O12   
2  isoniazid|54-85-3|Isonicotinic acid hydrazide|...  137.14      C6H7N3O   
3  pyrazinamide|pyrazinecarboxamide|98-96-4|pyraz...  123.11      C5H5N3O   
4  ciprofloxacin|85721-33-1|Ciprofloxacine|Ciprob...  331.34  C17H18FN3O3   

   polararea  complexity  xlogp  heavycnt  hbonddonor  ...  gpfamilycnt  \
0       64.4        92.9   -1.5         7           2  ...         5393   
1      220.0      1620.0    4.9        59           6  ...        17177   
2       68.0       120.0   -0.7        10           2  ...        34271   
3       68.9       115.0   -0.6         9           1  ...        10753   
4       72.9       571.0   -1.1        24           2  ...        26622   

   neighbortype   meshheadings  \
0         2D+3D    Cycloserine   
1            2D       Rifampin   
2         2D+3D      Isoniazid   
3         2D+3D   Pyrazinamide   
4         2D+3D  Ciprofloxacin   

                                           annothits annothitcnt  \
0  Biological Test Results|Interactions and Pathw...          18   
1  Biological Test Results|Interactions and Pathw...          18   
2  Biological Test Results|Interactions and Pathw...          18   
3  Biological Test Results|Interactions and Pathw...          18   
4  Biological Test Results|Interactions and Pathw...          18   

                                                aids  cidcdate  \
0  357|410|411|422|429|436|444|445|446|447|448|45...  20050601   
1  5|7|9|19|21|25|29|31|33|35|37|39|41|43|45|47|4...  20190115   
2  155|157|161|165|167|175|179|180|192|194|200|21...  20040916   
3  155|157|161|165|167|175|357|368|374|377|410|41...  20050325   
4  357|410|411|444|445|446|447|448|450|451|526|53...  20050325   

                                          sidsrcname  \
0  001Chemical|10X CHEM|3B Scientific (Wuhan) Cor...   
1  ABBLIS Chemicals|Abe Lab, University of Texas ...   
2  001Chemical|10X CHEM|3B Scientific (Wuhan) Cor...   
3  001Chemical|10X CHEM|3B Scientific (Wuhan) Cor...   
4  001Chemical|10X CHEM|3B Scientific (Wuhan) Cor...   

                                             depcatg  \
0  Chemical Vendors|Curation Efforts|Governmental...   
1  Chemical Vendors|Curation Efforts|Governmental...   
2  Chemical Vendors|Curation Efforts|Governmental...   
3  Chemical Vendors|Curation Efforts|Governmental...   
4  Chemical Vendors|Curation Efforts|Governmental...   

                                          annotation  
0  C254 - Anti-Infective Agent > C52588 - Antibac...  
1  C254 - Anti-Infective Agent > C52588 - Antibac...  
2  C254 - Anti-Infective Agent > C52588 - Antibac...  
3  D000890 - Anti-Infective Agents > D000900 - An...  
4  C254 - Anti-Infective Agent > C258 - Antibioti...  

[5 rows x 40 columns]

In [55]:
d6.shape

(165, 40)

In [56]:
d6.columns

Index([' cid', 'cmpdname', 'cmpdsynonym', 'mw', 'mf', 'polararea',
       'complexity', 'xlogp', 'heavycnt', 'hbonddonor', 'hbondacc', 'rotbonds',
       'inchi', 'isosmiles', 'canonicalsmiles', 'inchikey', 'iupacname',
       'exactmass', 'monoisotopicmass', 'charge', 'covalentunitcnt',
       'isotopeatomcnt', 'totalatomstereocnt', 'definedatomstereocnt',
       'undefinedatomstereocnt', 'totalbondstereocnt', 'definedbondstereocnt',
       'undefinedbondstereocnt', 'pclidcnt', 'gpidcnt', 'gpfamilycnt',
       'neighbortype', 'meshheadings', 'annothits', 'annothitcnt', 'aids',
       'cidcdate', 'sidsrcname', 'depcatg', 'annotation'],
      dtype='object')

In [57]:
d6 = d6[['isosmiles']]
d6.shape

(165, 1)

In [58]:
d6.rename({'isosmiles':'Smiles'}, axis = 1, inplace = True)
d6['Activity'] = 1

In [59]:
d6.shape

(165, 2)

In [60]:
d6.head()

Smiles  Activity
0                                 C1[C@H](C(=O)NO1)N         1
1  C[C@H]1/C=C/C=C(\C(=O)NC2=C(C(=C3C(=C2O)C(=C(C...         1
2                                 C1=CN=CC=C1C(=O)NN         1
3                                C1=CN=C(C=N1)C(=O)N         1
4   C1CC1N2C=C(C(=O)C3=CC(=C(C=C32)N4CCNCC4)F)C(=O)O         1

In [61]:
df = pd.concat([d1, d2, d3, d4, d5], ignore_index=True)
df.shape

(24043, 5)

In [62]:
if d1.shape[0] + d2.shape[0] + d3.shape[0] + d4.shape[0] + d5.shape[0] == df.shape[0]:
    print("Dataframes combined successfully.")
else:
    print("Dataframes combined unsuccessfully.")

Dataframes combined successfully.


In [63]:
df.head()

Smiles  Molecular Weight  \
0         O=C(O)c1cn(C2CC2)c2cc(N3CCNCC3)c(F)cc2c1=O            331.35   
1      O=[N+]([O-])c1ccc(/C=N/NC(=S)Nc2ccc(Cl)cn2)o1            325.74   
2  COc1ccc(NC(=S)N/N=C/c2ccc([N+](=O)[O-])o2)c(OC)c1            350.36   
3          O=[N+]([O-])c1ccc(/C=N/NC(=S)Nc2ccccc2)o1            290.30   
4    O=[N+]([O-])c1ccc(/C=N/NC(=S)Nc2ncc(Br)cc2Br)o1            449.08   

   Standard Value Standard Type Standard Units  
0            2.35           MIC             µM  
1           38.30           MIC             µM  
2           17.80           MIC             µM  
3           86.10           MIC             µM  
4            6.90           MIC             µM

In [64]:
df['Standard Units'].unique()

array(['µM'], dtype=object)

In [65]:
df_10 = assign_activity(df, 'Standard Value', 10, 0, 1)
df_10['Activity'].value_counts()

Activity
0    14837
1     9206
Name: count, dtype: int64

In [66]:
df_5 = assign_activity(df, 'Standard Value', 5, 0, 1)
df_5['Activity'].value_counts()

Activity
0    16987
1     7056
Name: count, dtype: int64

In [67]:
df_1 = assign_activity(df, 'Standard Value', 1, 0, 1)
df_1['Activity'].value_counts()

Activity
0    20319
1     3724
Name: count, dtype: int64

In [68]:
df_10.head()

Smiles  Molecular Weight  \
0         O=C(O)c1cn(C2CC2)c2cc(N3CCNCC3)c(F)cc2c1=O            331.35   
1      O=[N+]([O-])c1ccc(/C=N/NC(=S)Nc2ccc(Cl)cn2)o1            325.74   
2  COc1ccc(NC(=S)N/N=C/c2ccc([N+](=O)[O-])o2)c(OC)c1            350.36   
3          O=[N+]([O-])c1ccc(/C=N/NC(=S)Nc2ccccc2)o1            290.30   
4    O=[N+]([O-])c1ccc(/C=N/NC(=S)Nc2ncc(Br)cc2Br)o1            449.08   

   Standard Value Standard Type Standard Units  Activity  
0            2.35           MIC             µM         0  
1           38.30           MIC             µM         0  
2           17.80           MIC             µM         0  
3           86.10           MIC             µM         0  
4            6.90           MIC             µM         0

In [69]:
df_10.describe()

Molecular Weight  Standard Value      Activity
count      24043.000000    23148.000000  24043.000000
mean         410.911721      147.578379      0.154889
std          155.173016     1699.630833      0.361807
min           45.090000        0.000000      0.000000
25%          325.915000        4.269893      0.000000
50%          389.410000       22.000000      0.000000
75%          464.450000       85.403550      0.000000
max         2841.940000    76690.000000      1.000000

In [70]:
df_5.describe()

Molecular Weight  Standard Value      Activity
count      24043.000000    23148.000000  24043.000000
mean         410.911721      147.578379      0.154889
std          155.173016     1699.630833      0.361807
min           45.090000        0.000000      0.000000
25%          325.915000        4.269893      0.000000
50%          389.410000       22.000000      0.000000
75%          464.450000       85.403550      0.000000
max         2841.940000    76690.000000      1.000000

In [71]:
df_10.drop(['Standard Value', 'Standard Type', 'Standard Units', 'Molecular Weight'], axis=1, inplace=True)
df_10.shape

(24043, 2)

In [72]:
df_10 = df_10.drop_duplicates(subset='Smiles')
df_10.shape

(23627, 2)

In [73]:
df_10 = pd.concat([df_10, d6], ignore_index=True)
df_10.shape

(23792, 2)

In [74]:
df_10.drop_duplicates(subset = 'Smiles')
df_10.dropna(inplace = True)

In [75]:
df_10.head()

Smiles  Activity
0         O=C(O)c1cn(C2CC2)c2cc(N3CCNCC3)c(F)cc2c1=O         0
1      O=[N+]([O-])c1ccc(/C=N/NC(=S)Nc2ccc(Cl)cn2)o1         0
2  COc1ccc(NC(=S)N/N=C/c2ccc([N+](=O)[O-])o2)c(OC)c1         0
3          O=[N+]([O-])c1ccc(/C=N/NC(=S)Nc2ccccc2)o1         0
4    O=[N+]([O-])c1ccc(/C=N/NC(=S)Nc2ncc(Br)cc2Br)o1         0

In [76]:
df_10.shape

(23791, 2)

In [77]:
df_10.to_csv('data.csv', index=False)